In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pyedflib
import seaborn
import numpy as np
import mne
import neurokit2 as nk
import os
import seaborn as sns
from scipy.fftpack import fft
from scipy.interpolate import interp1d

In [ ]:
def compute_CSI_CVI(RR, t_RR, wind):
    """
    Compute CSI and CVI indices from RR intervals. Code source: https://github.com/diegocandiar/robust_hrv/blob/main/compute_CSI_CVI.m

    Parameters
    ----------
    RR : array_like
        Array of R-R intervals.
    t_RR : array_like
        Corresponding time stamps for the RR intervals.
    wind : float
        Window length (in same units as t_RR) for the time-varying calculation.

    Returns
    -------
    CSI_out : numpy.ndarray
        Interpolated CSI time series.
    CVI_out : numpy.ndarray
        Interpolated CVI time series.
    t_out : numpy.ndarray
        Time stamps corresponding to the output series.
    """
    # Sampling frequency
    Fs = 4

    # Create time vector from first to last t_RR with spacing 1/Fs
    time = np.arange(t_RR[0], t_RR[-1] + 1/Fs, 1/Fs)

    # -----------------------------
    # First Poincare Plot Calculation
    # -----------------------------
    sd = np.diff(RR)
    # Use sample standard deviation (ddof=1) to mimic MATLAB's std behavior
    sd_std = np.std(sd, ddof=1)
    RR_std = np.std(RR, ddof=1)
    SD01 = np.sqrt(0.5 * (sd_std ** 2))
    SD02 = np.sqrt(2 * (RR_std ** 2) - (0.5 * (sd_std ** 2)))

    # -----------------------------
    # Time-varying SD Calculation
    # -----------------------------
    t1 = time[0]
    t2 = t1 + wind
    # Find indices where t_RR is greater than t2
    ixs = np.where(t_RR > t2)[0]
    nt = len(ixs) - 1  # number of segments (MATLAB: length(ixs)-1)

    # Preallocate arrays for SD1, SD2 and the center time t_C
    SD1 = np.zeros(nt)
    SD2 = np.zeros(nt)
    t_C = np.zeros(nt)

    for k in range(nt):
        i = ixs[k]
        t2_val = t_RR[i]
        t1_val = t_RR[i] - wind
        # Find indices for the current window [t1_val, t2_val]
        ix = np.where((t_RR >= t1_val) & (t_RR <= t2_val))[0]

        # Compute differences for the current window
        sd_local = np.diff(RR[ix])
        # Check to avoid issues with too few points
        local_sd_std = np.std(sd_local, ddof=1) if len(sd_local) > 1 else np.nan
        local_RR_std = np.std(RR[ix], ddof=1) if len(RR[ix]) > 1 else np.nan

        SD1[k] = np.sqrt(0.5 * (local_sd_std ** 2)) #approximate approach
        SD2[k] = np.sqrt(2 * (local_RR_std ** 2) - (0.5 * (local_sd_std ** 2)))
        t_C[k] = t2_val


    # Normalize the time-varying SDs to match the global values
    SD1 = SD1 - np.mean(SD1) + SD01
    SD2 = SD2 - np.mean(SD2) + SD02

    # -----------------------------
    # Compute CVI and CSI
    # -----------------------------
   
    CVI = SD1 * 10
    CSI = SD2 * 10

    # -----------------------------
    # Remove duplicates in t_C
    # -----------------------------
    t_C, unique_indices = np.unique(t_C, return_index=True)
    CVI = CVI[unique_indices]
    CSI = CSI[unique_indices]

    # -----------------------------
    # Interpolation to Create Output Time Series
    # -----------------------------
    t_out = np.arange(t_C[0], t_C[-1] + 1/Fs, 1/Fs)

    # Using cubic spline interpolation (equivalent to MATLAB's 'Spline')
    interp_CVI = interp1d(t_C, CVI, kind='cubic', fill_value="extrapolate")
    interp_CSI = interp1d(t_C, CSI, kind='cubic', fill_value="extrapolate")

    CVI_out = interp_CVI(t_out)
    CSI_out = interp_CSI(t_out)

    return CSI_out, CVI_out, t_out